In [1]:
#This code assumes you have a syntaxnet/Parsey McParseface server already running on "url" address
#Check this repository for instructions on how to do so: https://hub.docker.com/r/andersrye/parsey-universal-server

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
from urllib.parse import urlencode
from urllib.request import Request, urlopen
import json
import os
import spacy
import re
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook
import os.path
import time
def processText(text="",chunkSize=50):
    url = 'http://ENG401347.boisestate.edu:7777' # Set destination URL here
    #url = 'http://0.0.0.0:7777'
    result = []
    sentences = text.split("\n")
    for i in range(0, len(sentences), chunkSize):
        print(i,"/",len(sentences))
        chunk=  "\n".join(sentences[i:i + chunkSize])
        #print(chunk)
        if len(chunk.replace("\n","").replace(" ",""))>0:
            request = Request(url, chunk.encode(),headers={"Content-Type":"text/plain"})
            jsonresponse = urlopen(request,timeout=240).read().decode()
            tmp = json.loads(jsonresponse)
            result= result + tmp
    return result

def loadDocsFromFolder(folder=""):
    allDocs = {}
    for root, subFolders, files in os.walk(folder):
        for filename in files:
            text= open(root+"/"+filename, 'r', encoding="utf8").read()
            allDocs[filename]=text
    return allDocs
def removeCitations(text=""):
    text= re.sub(r'\[\d+\]', '', text)
    return text

class Tokenizer:
    def __init__(self,lang="en"):
        """
        :param config:
        """

        self.spacy_nlp = spacy.load('en')

    def tokenize(self, input):
        return [x.text for x in self.spacy_nlp.tokenizer(input) if x.text != " "]

    def split_sentences(self, input):
        return [x.text for x in self.spacy_nlp(input).sents if x.text != " "]
tokenizer = Tokenizer()

In [3]:
def prepareTextForSyntaxnet(text=""):
    tmp = removeCitations(text)
    tmp = tmp.replace("\n","")
    result = ""
    for sentence in tokenizer.split_sentences(tmp):
        tokenized_sentence = " ".join(tokenizer.tokenize(sentence))
        if(len(tokenized_sentence.replace(" ","")))>8:
            result = result + tokenized_sentence + "\n"
    return result

In [4]:
%%time
for i in range(10):
    %time processText("this is a text"*100)

0 / 1
Wall time: 3.27 s
0 / 1
Wall time: 3.35 s
0 / 1
Wall time: 3.52 s
0 / 1
Wall time: 3.41 s
0 / 1
Wall time: 3.44 s
0 / 1
Wall time: 3.52 s
0 / 1
Wall time: 3.79 s
0 / 1
Wall time: 3.45 s
0 / 1
Wall time: 3.95 s
0 / 1
Wall time: 3.84 s
Wall time: 35.6 s


In [4]:

rawFolder="../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/raw/"
syntaxnetFolder="../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/"
dfolder = "vikidia/en/"
try:  
    os.makedirs(syntaxnetFolder+dfolder)
except:
    pass

docs= loadDocsFromFolder(rawFolder+ dfolder)

In [5]:
for key in tqdm_notebook(docs):
    docs[key] = prepareTextForSyntaxnet(docs[key])

In [6]:
ignoreDocNames = ["Albrecht_D%C3%BCrer","American_history","Ancient_Greece","Computer","#Convention_on_the_Rights_of_the_Child"]
ignoreDocNames=[]

In [7]:
maxSentences = 0
for key in tqdm_notebook(docs):
    #print(docs[key])
    outFilename =syntaxnetFolder+dfolder+key+'.syntaxnet.json'
    
    if key in ignoreDocNames:
        continue
    maxSentences = max(maxSentences,docs[key].count("\n"))
    if not os.path.exists(outFilename) :
        #try:
            
            jsonResult = processText(docs[key])
            
            with open(outFilename, 'w') as outfile:
                json.dump(jsonResult, outfile, indent=4)
            print("CREATED:"+outFilename)
        #except:
        #    print("ERROR:"+outFilename)
        #    pass
    else:
        print("FOUND:"+outFilename)
    #time.sleep(1)

0 / 6
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/$.syntaxnet.json
0 / 4
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/%C3%89douard_Manet.syntaxnet.json
0 / 2
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/%CE%A0.syntaxnet.json
0 / 17
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/19th_Century_French_Firearms.syntaxnet.json
0 / 1
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/2008.syntaxnet.json
0 / 3
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/2013.syntaxnet.json
0 / 3
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/2014.syntaxnet.json
0 / 3
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/e

CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/Apple_(fruit).syntaxnet.json
0 / 3
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/Apple_Computer.syntaxnet.json
0 / 3
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/Apple_Computer,_Inc.syntaxnet.json
0 / 3
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/Apple_Inc.syntaxnet.json
0 / 11
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/Apple_pie.syntaxnet.json
0 / 5
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/April.syntaxnet.json
0 / 27
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vikidia/en/Arabic.syntaxnet.json
0 / 27
CREATED:../data/readabilityCorpora/6lang/VikidiaVsWikipedia6LangAndParallel/syntaxnet/vik

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host